Installation des librairies

In [1]:
import sys
if('numpy' not in sys.modules):
    !pip install numpy
if('matplotlib' not in sys.modules):
    !pip install matplotlib
if('sympy' not in sys.modules):
    !pip install sympy
if('consolemenu' not in sys.modules):
    !pip install console-menu

Les importations

In [2]:
import numpy as np
from sympy import symbols, Matrix, Function, simplify, exp, hessian, solve, init_printing, sympify , var,lambdify
import sympy as sym
import re
from consolemenu import SelectionMenu,format
import matplotlib.pyplot as plt
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D

Les variables globales

In [3]:
#declarer les symboles/variables
x,y,z,t,w = var('x y z t w')
xn = np.array([x,y,z,t,w])

#nombre des variables (max 5)
n = 0

#la fonction
f = None

#gradiant de la fonction
grad = None

#matrice hessian de la fonction
hessianM = None

#marge d'erreur ou la prècision
eps = 0

#pas de depart de la regle d'armijo
alpha0 = 0

#vecteur de depart de la methode de gradiant conjugée
x0 = None

#la courbe 3D
courbe = None

#les axes (valeurs) de la courbe 3D
X = None
Y = None
Z = None

#parametre de la regle d'armijo
beta = 0.9

#Valeurs des xi (iterations de gradiant conjugée) pour visualiser le comparatif entre 3 differents X0
Vx = []
Vy = []

#nombres des figures (courbes)
nbrFigures = 0

#valeur considerée comme infinie
INF = -2000000

#nombre des iterations max
iterationsMax = 2000

Calculer gradiant

In [4]:
def calculer_gradiant(f):
    F = Matrix([f])
    gradf = simplify(F.jacobian(xn))
    gradiant = [None] * n
    i = 0
    while i < n:
        gradiant[i] = gradf[i]
        i += 1
    return np.array(gradiant)

Afficher gradiant

In [5]:
def afficherGrad(gradiant):
    i = 0
    print(bcolors.BOLD,"* Gradiant(f) = ",bcolors.ENDC)
    while(i < n):
        print("[(",i+1,")",gradiant[i],"]")
        i += 1

Calculer matrice hessian

In [6]:
def calculer_hessian(f):
    global hessianM
    f1 = f.copy()
    F = Matrix([f1])
    i = 0
    hessianf = simplify(hessian(F,xn))
    hessianM = np.array(hessianf)
    return hessianM

Afficher matrice hessian

In [7]:
def afficherHessian(Mhessian):
    i = 0
    print(bcolors.BOLD,"* Hessian(f) = ",bcolors.ENDC)
    while(i < n):
        j = 0
        print("[",end='')
        while(j < n):
            print("(",i+1,")(",j+1,") ",Mhessian[i][j],end='')
            print("     ",end='')
            j += 1
        print("]")
        i += 1

1er fonction de la memoire

In [8]:
def rosenbrock():
    global n,grad
    n = 2
    f = (1-x)**2 + 100*(y - x**2)**2
    grad = calculer_gradiant(f)
    return f

2eme fonction de la memoire

In [9]:
def fct2():
    global n,grad
    n = 2
    f = x*exp(-x**2 - y**2)
    grad = calculer_gradiant(f)
    return f

3eme fonction de la memoire

In [10]:
def fct3():
    global n,grad
    n = 5
    A = np.array([[3, -1, 0,0,0], [-1, 12, -1,0,0], [0, -1, 24,-1,0] , [0, 0, -1,48,-1] , [0, 0, 0,-1,96]])
    b = np.array([1,2,3,4,5])
    f = 1/2*np.dot(np.matmul(A,xn),xn) - np.dot(b,xn)
    grad = calculer_gradiant(f)
    return f

Remplacer les variables de f par les valeurs dans Vx

In [11]:
def valeurF(f,V):
    i = 0
    f1 = f.copy()
    while i < n:
        f1 = f1.subs({xn[i]:V[i]})
        i += 1
    return f1

Remplacer les variables du vecteur grad par les valeurs dans Vx

In [12]:
def valeurGrad(grad,V):
    i = 0
    gradValeur = grad.copy()
    while i < n:
        j = 0
        while j < n:
            gradValeur[i] = gradValeur[i].subs({xn[j]:V[j]})
            j += 1
        i += 1
    return np.array(gradValeur,dtype=np.float64)

Calculer le gradiant conjugée 

In [13]:
def GC_armijo(f, grad,alpha_0,x0, eps,beta):
    global Vx,Vy
    
    iteration = 0
    xi = x0
    f_x = valeurF(f,xi)
    grad_x0 = valeurGrad(grad,xi)
    grad_x = valeurGrad(grad,xi)
    d_x = - grad_x
    normGrad = np.linalg.norm(grad_x)
    
    while normGrad > eps :
        iteration = iteration + 1    
        alpha = alpha_0
        # Calcul pas d'armijo
        alpha = armijo_step(alpha,xi,f,f_x,grad_x0,d_x,beta)
        # Calcul point successeur
        xi = xi + alpha * d_x
        # Calcul gradient point successeur
        grad_x = valeurGrad(grad,xi)
        f_x = valeurF(f,xi)
        # Calcul beta de conjugaison
        betaCJ = ((np.linalg.norm(grad_x)**2)/(np.linalg.norm(grad_x0)**2))
        # Calcul la nouvelle direction
        d_x = betaCJ * d_x - grad_x
        grad_x0 = grad_x
        normGrad = np.linalg.norm(grad_x)
        
        if(n == 2):
            Vx.append(xi[0])
            Vy.append(xi[1])
        elif(n == 1):
            Vx.append(xi[0])
        if(f_x < INF or iteration > iterationsMax):
            break
        
    return xi,f_x,iteration

Calculer le pas d'armijo

In [14]:
def armijo_step(alpha_0,xi,f,f_x,grad_x,d_x,beta):
    # La methode commence par alpha_0
    alpha = alpha_0
    f0 = f_x
    f1 = valeurF(f,xi+alpha*d_x)
    # Calcul la pas alpha avec la condition d'armijo
    if (f1<f0+alpha*np.dot(grad_x,d_x)) :
        while (f1<f0+alpha*beta*np.dot(grad_x,d_x)): 
            # Agrandir la pas en le multipliant par 2
            alpha = alpha*2.0
            f1 = valeurF(f,xi+alpha*d_x)
            
    else :
        while (f1>f0+alpha*beta*np.dot(grad_x,d_x)) :
             # Réduire la pas en le divise par 2
            alpha=alpha/2.0
            f1 = valeurF(f,xi+alpha*d_x) 
    return alpha

Tracer la courbe

In [15]:
def tracerCourbe(f):
    
    print("Entrer le debut du domaine : ")
    start = lireFloat()
    print("Entrer la fin du domaine : ")
    end = lireFloat()
    
    global X,Y,Z,nbrFigures,courbe
    
    f1 = lambdify([x,y],f, 'numpy')
    
    #definir les domaines de x1 et x2
    dx = np.linspace(start,end,50)
    dy = np.linspace(start,end,50)
    
    #Créer un grid qui contient tous les valeurs de x1 et x2
    X,Y = np.meshgrid(dx,dy)
    
    #Creer une variable pour stocker les resultats
    Z = f1(X,Y)
    
    #Pour une visualisattion interactive où on peut faire pivoter la figure en 3 dimensions
    %matplotlib notebook
    
    #Visualisation
    nbrFigures += 1
    plt.figure(nbrFigures)
    courbe = plt.axes(projection = '3d')
    courbe.plot_surface(X,Y,Z,rstride = 1,cstride = 1,cmap='cool',edgecolor='none',alpha=0.7)
    courbe.set_title("Courbe",fontsize = 13)
    courbe.set_xlabel('X', fontsize = 11)
    courbe.set_ylabel('Y', fontsize = 11)
    courbe.set_zlabel('Z', fontsize = 11)
    plt.show()

Ajouter les lignes de niveaux au graphe existant

In [16]:
def lignesNiveaux():
    global courbe
    if(courbe == None):
        tracerCourbe(f)
    lignesNiveaux = courbe.contour(X,Y,Z,10,colors='black',linewidths = 1)

Tracer les iterations

In [17]:
def tracerIterations():
    global Vx,Vy,nbrFigures
    
    #creer une nouvelle figure
    nbrFigures += 1
    plt.figure(nbrFigures)
    #Axe x1
    plt.xlabel("X")
    #Titre de la figure
    plt.title("Les iterations de l'algorithme 'Gradient conjugué avec la règle d’Armijo'")
    if(n == 1):
        Vx.insert(0,x0[0])
        plt.plot(Vx1,'bo',label = "Iterations")
        plt.plot(Vx1[0],'ro',label = "Iterations")
        plt.plot(Vx1[len(Vx1) - 1],'yo',label = "Solution")
    elif(n == 2):
        Vx.insert(0, x0[0])
        Vy.insert(0, x0[1])
        plt.plot(Vx,Vy,'bo',label = "Iterations")
        plt.plot(Vx[0],Vy[0],'ro',label = f"X0 {x0}")
        plt.plot(Vx[len(Vx) - 1],Vy[len(Vy) - 1],'yo',label = "Solution")
        plt.ylabel("Y")
    
    Vx = []
    Vy = []
        
    plt.legend()
    plt.show()
        
    

Visualiser un comparatif avec 3 différents X0 (le vecteur de départ)

In [18]:
def tracerComparatif():
    global Vx,Vy,nbrFigures
    
    #lire les x0
    xn = []
    couleurs = ['ro','go','bo']
    i = 0
    while(i < 3):
        j = 0
        X0 = []
        print(bcolors.BOLD,"---> Entrer comparatif X0(",i+1,") : ",bcolors.ENDC)
        while j < n:
            print("Entrer X0(",i+1,")[",j+1,"] : ",end = '')
            X0.append(lireFloat())
            j += 1
        xn.append(X0)
        i += 1
    
    i = 0
    
    #créer une nouvelle figure
    nbrFigures += 1
    plt.figure(nbrFigures)
    
    #executer l'alorithme du gradiant conjugée pour les 3 x0 et tracer chaque fois les xi (les iterations)
    while(i < 3):
        Vx = []
        Vx.append(xn[i][0])
        
        if(n == 2):
            Vy = []
            Vy.append(xn[i][1])
        print(bcolors.OKBLUE,"En cours de calculer avec X0(",i+1,") ...",bcolors.ENDC)
        GC_armijo(f, grad, alpha0, xn[i], eps, beta)
        if(n == 1):
            plt.plot(Vx,couleurs[i],label = xn[i])
        elif(n == 2):
            plt.plot(Vx,Vy,couleurs[i],label = xn[i])
        i += 1
        
    plt.xlabel("X")
    if(n == 2):
        plt.ylabel("Y")
    plt.title("Comparaison entre 3 X0")
    plt.legend()
    plt.show()
    Vx = []
    Vy = []

Lire entier

In [19]:
def lireEntier()-> int:
    
    test = False
    while not test:
        nb_var = input()
        try:
            nb_var = int(nb_var)
            return  nb_var
        except Exception as e:
            print(bcolors.FAIL,"Veuillez saisir un entier valide ",bcolors.ENDC)

Lire float

In [20]:
def lireFloat()-> float:
    
    test = False
    while not test:
        nb_var = input()
        try:
            nb_var = float(nb_var)
            return  nb_var
        except Exception as e:
            print(bcolors.FAIL,"Veuillez saisir un flottant valide ",bcolors.ENDC)

In [21]:
#fonction pour récupérer la fonction saisie par l'utilisateur
def functionsaisie(ch): 
    global fctt
    fctt=ch

Lire fonction

In [22]:
def lireFonction():
    global grad
    xn1,xn2,xn3,xn4,xn5 = var('x y z t w')
    while True:
####################cette partie est exécutée dans l'interface####################
#         print(bcolors.BOLD,"---> Veuillez saisir une fonction valide : ",bcolors.ENDC)
#         print(bcolors.WARNING,"!!!Remarque : les variables sont notées x,y,z,t,w et la puissance x^2 est notée x**2",bcolors.ENDC)
#         print(bcolors.OKBLUE,"Exemple : (x**2 - 5 + y**2 + (x**2 -1)*(y**2 - 4))/(x**2 + y**2 + 1)**2",bcolors.ENDC)
#           f1 = input()  ############### à modifier
        f1 = fctt 
        try:
            f1 = sympify(f1)#f1=
            f2 = f1.copy()
        except Exception as e:
            continue
        
        f2 = f2.subs({xn1:1.0,xn2:1.0,xn3:1.0,xn4:1.0,xn5:1.0})     
        if(type(f2) is not sym.core.numbers.Float and type(f2) is not sym.core.numbers.Zero):
            continue
        else:
            break
            
    grad = calculer_gradiant(f1)
    return f1

Lire les parametres de la fonction

In [23]:
def lireParametres():
    n=2
    global eps,alpha0,x0
    while(eps < 0.0001 or eps > 0.1):
        print(bcolors.BOLD,"---> Entrer la precision (0.0001 <= eps <= 0.1): ",bcolors.ENDC,end = '')
        eps = lireFloat()
    while(alpha0 <= 0):
        print(bcolors.BOLD,"---> Entrer le pas de départ de la règle d’Armijo (alpha0 > 0): ",bcolors.ENDC,end = '')
        alpha0 = lireFloat()
    print(bcolors.BOLD,"---> Entrer le vecteur de depart (X0): ",bcolors.ENDC)
    i = 0
    x0 = [None] * n
    while i < n:
        print("X0[",i+1,"] : ",end = '')
        x0[i] = lireFloat()
        i += 1

Reinitialiser les variables globales

In [24]:
def reinitialiser():
    global n,f,grad,hessianM,eps,alpha0,x0,courbe,X,Y,Z,Vx,Vy
    
    #nombre des variables (max 5)
    n = 2
    
    #la fonction
    f = None
    
    #gradiant de la fonction
    grad = None
    
    #matrice hessian de la fonction
    hessianM = None
    
    #marge d'erreur ou la prècision
    eps = 0
    
    #pas de depart de la regle d'armijo
    alpha0 = 0
    
    #vecteur de depart de la methode de gradiant conjugée
    x0 = None
    
    #la courbe 3D
    courbe = None
    
    #les axes (valeurs) de la courbe 3D
    X = None
    Y = None
    Z = None
    
    #Valeurs des xi (iterations de gradiant conjugée) pour visualiser le comparatif entre 3 differents X0
    Vx = []
    VY = []
    

Enregistrer la resultat dans un fichier texte 'Resultats.txt' qui est dans le meme emplacement que le programme

In [25]:
def enregistrer(res):
    try:
        file = open('Resultats.txt', 'a')
        file.write(f"* La fonction est : {f}\n")
        file.write(f"* La solution (xmin) = {res[0]}\n")
        file.write(f"* Valeur minimal de f (fmin) = {res[1]}\n")
        file.write(f"* Nombre des iterations effectués = {res[2]}\n")
        file.write(f"* Vecteur de depart (x0) = {x0}\n")
        file.write(f"* Pas de depart (alpha0) = {alpha0}\n")
        file.write(f"---------------------------------------------------\n")
        file.close()
        print(bcolors.OKBLUE,"Le résultat est enregistré avec succès dans 'Resultats.txt'",bcolors.ENDC)
    except Exception as e:
        print(bcolors.FAIL,"Le résultat n'a pas été enregistré",bcolors.ENDC)

In [26]:
#menu1(projets1,2&4)
def myfunction(ch):
    global choix
    if(ch=='choisir'):
        choix=1
    elif (ch=='saisir'):
        choix=2
    else: 
        choix=3

Menu 1

In [27]:
def menu1():
    #reinitialiser les variables globales
    reinitialiser()
    
    global n,f
####################cette partie est exécutée dans l'interface####################
#     #liste des options
#     a_list=["Choisir une fonction de la memoire", "Saisir votre propre fonction"];
#     menu = SelectionMenu(a_list,)
    
#     #exit
#     menu.exit_item.text="Quitter";
    
#     #border
#     menu.formatter.set_border_style(format.HeavyOuterLightInnerBorderStyle())
    
#     #titre
#     menu.title = "Bienvenue au projet 2"
    
#     #indication
#     menu.subtitle = "Veuillez nous donner l'option que vous recherchez:"
    
#     #executer le menu1
#     menu.show()
#     menu.join()
    
#     #option selectionnée
#     choix = menu.selected_option
    
    if choix == 1:
        menuFonctionMemoire()
        menu2()
    elif choix == 2:
        
####################cette partie est exécutée dans l'interface####################
        #print(bcolors.BOLD,"---> Entrer le nombres de variables : ",bcolors.ENDC,end = '')
        n = 2
#         while(n < 2 or n > 2):
#             n = lireEntier()
        f = lireFonction()
        #lireParametres()
        menu2()
    elif choix == 3:
        print("Fin du programme")
        sys.exit("Fin du programme") #pour migrer du console vers l'interface

In [28]:
#fonction pour récupérer la fonction choisie par l'utilisateur de la mémoire
def myfunction2(ch): 
    global choix2
    if(ch=='(1 − 𝑥)² + 100(𝑦 − 𝑥²)²'):
        choix2=1
    elif (ch=='xexp(-x²-y²)'):
        choix2 =2
    else: 
        choix2=3

Menu de choisir une fonction de la memoire

In [29]:
def menuFonctionMemoire():
####################cette partie est exécutée dans l'interface####################
#     #liste des options
#     a_list=["f(x,y) = (1 − x)^2 + 100(y − x^2)^2",
#             "f(x,y) = x*exp(-x^2 - y^2)",
#             "f(x) = 0.5 <Ax,x> − <b,x> \n A = [3,-1,0,0,0]\n\t[-1,-12,-1,0,0]\n\t[0,-1,-24,-1,0]\n\t[0,0,-1,-48,-1]\n\t[0,0,0,-1,96]]\n b =[1,2,3,4,5] \n x = [x,y,z,t,w]"];
    
#     menu = SelectionMenu(a_list,"Saisir l'une des 2 fonction:")
    
#     #exit
#     menu.exit_item.text="Revenir au Menu principal";
    
#     #border
#     menu.formatter.set_border_style(format.HeavyOuterLightInnerBorderStyle())
    
#     #titre
#     menu.title="Bienvenue au projet 2"
    
#     #indication
#     menu.subtitle="Veuillez nous donner l'option que vous recherchez :"
    
#     #executer menu des fonctions memoire
#     menu.show()
#     menu.join()
    
#     selection = menu.selected_option
    global f
    if choix2 == 1:
        f = rosenbrock()
        lireParametres()
        menu2()
    elif choix2 == 2:
        f = fct2()
        lireParametres()
        menu2()
    elif choix2 == 3:
        f = fct3()
        lireParametres()
        menu2()
    #elif choix2 == 4:
     #   menu1()

Menu 2

In [30]:
def menu2():
    
    #liste des options
    a_list=["Tracer la courbe",
            "Tracer les lignes de niveaux et les ajouter au graphe existant",
            "Calculer le vecteur gradient",
            "Calculer la matrice Hessienne",
            "Appliquer la méthode de gradient conjugué avec la règle d’Armijo",
            "Visualiser un comparatif avec 3 différents X0"
            ];
    menu = SelectionMenu(a_list,"Donnez votre choix:")
    
    #exit menu
    menu.exit_item.text = "Revenir à linterface";
    
    #border
    menu.formatter.set_border_style(format.HeavyOuterLightInnerBorderStyle())
    
    #executer menu2
    menu.show()
    menu.join()
    
    #option selectionnée
    selection = menu.selected_option
    
    if selection == 0:
        #tracer la courbe
        if(n <= 2):
            tracerCourbe(f)
            print(bcolors.WARNING,"!!! La courbe s'affiche après la fin du programme",bcolors.ENDC)
        else:
            print(bcolors.FAIL,"Ne peut pas représenter graphiquement lorsque le nombre de variables est supérieur à 2",bcolors.ENDC)
        menu2()
        
    elif selection == 1:
        #tracer les lignes de niveaux et les ajouter au graphe existant
        if(n <= 2):
            lignesNiveaux()
            print(bcolors.WARNING,"!!! La courbe s'affiche après la fin du programme",bcolors.ENDC)
        else:
            print(bcolors.FAIL,"Ne peut pas représenter graphiquement lorsque le nombre de variables est supérieur à 2",bcolors.ENDC)
        menu2()
            
    elif selection == 2:
        #calculer le vecteur gradient
        afficherGrad(grad)
        menu2()
    elif selection == 3:
        #calculer la matrice Hessienne
        afficherHessian(calculer_hessian(f))
        menu2()
    elif selection == 4:
        #appliquer la méthode de gradient conjugué avec la règle d’Armijo
        lireParametres()
        print(bcolors.OKBLUE,"En cours de calculer ...",bcolors.ENDC)
        res = GC_armijo(f, grad, alpha0, x0, eps, beta)
        if(res[1]<INF):
            print(bcolors.FAIL,"Cette fonction tend vers l'infinie (-∞)",bcolors.ENDC)
        elif(res[2] > iterationsMax):
            print(bcolors.FAIL,":( l'algorithme n'a pas pu trouver le minimun, essayer avec une autre alpha0",bcolors.ENDC)
        else:
            print(bcolors.BOLD,"* La solution (xmin) = ",bcolors.ENDC,bcolors.OKCYAN,res[0],bcolors.ENDC)
            print(bcolors.BOLD,"* Valeur minimal de f (fmin) = ",bcolors.ENDC,bcolors.OKCYAN,res[1],bcolors.ENDC)
            print(bcolors.BOLD,"* Nombre des iterations effectués = ",bcolors.ENDC,bcolors.OKCYAN,res[2],bcolors.ENDC)
            print(bcolors.BOLD,"* Vecteur de depart (x0) = ",bcolors.ENDC,bcolors.OKCYAN,x0,bcolors.ENDC)
            print(bcolors.BOLD,"* Pas de depart (alpha0) = ",bcolors.ENDC,bcolors.OKCYAN,alpha0,bcolors.ENDC)
            enregistrer(res)
            if(n <= 2):
                tracerIterations()
                print(bcolors.WARNING,"!!! La courbe s'affiche après la fin du programme",bcolors.ENDC)
        menu2()
    elif selection == 5:
        #Visualiser un comparatif avec 3 différents X0
        if(n <= 2):
            tracerComparatif()
            print(bcolors.WARNING,"!!! La courbe s'affiche après la fin du programme",bcolors.ENDC)
        else:
            print(bcolors.FAIL,"Ne peut pas représenter graphiquement lorsque le nombre de variables est supérieur à 2",bcolors.ENDC)
        menu2()
            
    elif selection == 6:
        #menu1()
        menu1.exit_item.text="Revenir à linterface";
        #sys.exit('byee') #pour migrer du console vers l'interface

Les styles de text de la fonction print

In [31]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

Main

In [32]:
#menu1()